In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import lxml
from pandas import ExcelWriter
import xlsxwriter
from datetime import datetime

/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [6]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(chrome_options=options)

driver.implicitly_wait(1)
driver.maximize_window()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


WebDriverException: Message: Service chromedriver unexpectedly exited. Status code was: 127


In [ ]:
driver.get("http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701")
driver.find_element_by_css_selector('#grid1_4').click()
driver.find_element_by_css_selector('#grid1_7').click()
driver.find_element_by_css_selector('#grid1_8').click()
driver.find_element_by_css_selector('#grid1_9').click()
driver.find_element_by_css_selector('#grid1_10').click()
driver.find_element_by_css_selector('#grid1_11').click()

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', {'class':'us_table_ty1 h_fix zigbg_no'})


screenshot_name = "/root/.jupyter/financial_analysis/my_screenshot_name.png"
driver.save_screenshot(screenshot_name)

In [ ]:
table = str(table)

In [ ]:
type(table)

In [ ]:
pd.read_html(table)[0].transpose().reset_index()

# 이제 자동화 해본당


In [ ]:
kospi = pd.read_excel('/root/.jupyter/financial_analysis/data/kospi.xls')

In [ ]:
stock_code_list = []
def two_len(code):
    code = '0000' + code
    return code

def three_len(code):
    code = '000' + code
    return code

def four_len(code):
    code = '00' + code
    return code
    
def five_len(code):
    code = '0' + code
    return code

for i in range(0, len(kospi['종목코드'])):
    if(len(str(kospi['종목코드'][i])) == 4):
        stock_code_list.append([four_len(str(kospi['종목코드'][i])), kospi['기업명'][i]])
    elif(len(str(kospi['종목코드'][i])) == 5):
        stock_code_list.append([five_len(str(kospi['종목코드'][i])), kospi['기업명'][i]])
    elif(len(str(kospi['종목코드'][i])) == 3):
        stock_code_list.append([three_len(str(kospi['종목코드'][i])), kospi['기업명'][i]])
    elif(len(str(kospi['종목코드'][i])) == 2):
        stock_code_list.append([two_len(str(kospi['종목코드'][i])), kospi['기업명'][i]])
    else:
        stock_code_list.append([str(kospi['종목코드'][i]), kospi['기업명'][i]])
    
    

In [ ]:
total_code = []
for i in range(len(stock_code_list)):
    code = 'A' + stock_code_list[i][0]
    total_code.append(code)

In [ ]:
def state_of_comprehensive_income(code):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(chrome_options=options)

    driver.implicitly_wait(0.5)
    
    url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode='+code+'&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
    driver.get(url)
    driver.find_element_by_css_selector('#grid1_4').click()
    driver.find_element_by_css_selector('#grid1_7').click()
    driver.find_element_by_css_selector('#grid1_8').click()
    driver.find_element_by_css_selector('#grid1_9').click()
    driver.find_element_by_css_selector('#grid1_10').click()
    driver.find_element_by_css_selector('#grid1_11').click()
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'class':'us_table_ty1 h_fix zigbg_no'})
    state_of_comprehensive_income_df = pd.read_html(str(table))[0].transpose().reset_index()
    
    driver.close()
    return state_of_comprehensive_income_df

In [ ]:
def make_excel(df, company):
    file_name = '/root/.jupyter/financial_analysis/data/state_of_comprehensive_income/'+company+'.xlsx'
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='company_list')
    writer.close()

In [ ]:
for i in range(len(total_code)):
    try:

        df = state_of_comprehensive_income(total_code[i])
        make_excel(df, kospi['기업명'][i])
    except :
        print('Error')

In [ ]:
df = state_of_comprehensive_income("A000660")

In [ ]:
df